In [1]:
import pandas as pd
import numpy as np
import re
import json
import xlsxwriter
import glob
import xlrd
from datetime import datetime
pd.options.mode.chained_assignment = None

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Use Python 3.9.15

In [11]:
# Path to the folder containing CSV files
folder_path = f"/Users/arielgraham/Documents/Infovesta Tasks/trequity-2/2020-2025/*.csv"  # Adjust this path

# Get list of all CSV file paths
csv_files = glob.glob(folder_path)

# Read and combine all CSVs
df_list = [pd.read_csv(file) for file in csv_files]

df_all = pd.concat(df_list, ignore_index=True)[['portid','portdate','opening','high','low','closing','valuess', 'volume']]
df_all['opening'] = df_all['opening'].astype('float32')
df_all['high'] = df_all['high'].astype('float32')
df_all['low'] = df_all['low'].astype('float32')
df_all['closing'] = df_all['closing'].astype('float32')
df_all['valuess'] = pd.to_numeric(df_all['valuess'], downcast='integer')  # Convert to smallest int type
df_all['volume'] = pd.to_numeric(df_all['volume'], downcast='integer')  # Convert to smallest int type

df_all.head(10)

ValueError: No objects to concatenate

In [4]:
df_all = df_all.sort_values(['portid','portdate'],ascending=[True,True])
print(f"df_all: {np.round(df_all.memory_usage(deep=True).sum()/1e9,2)} Gigabytes")

df_all: 0.42 Gigabytes


In [9]:
df_all['portdate'].apply(lambda x: pd.to_datetime(x).year)

KeyboardInterrupt: 

In [ ]:
# path = "/Users/irfanhilman/Downloads/Database/Daily Stock Price Investdata"

# filenames = glob.glob(path + "/*.xls")
# print('File names:', filenames)

In [ ]:
def augment_investdata(data,file_name):
    outputxlsx = []
    for file in file_name:
        date_ = pd.read_excel(file,nrows=0)
        date_ = date_.columns[0][19:]
        date_ = pd.to_datetime(date_, format='%d-%b-%Y').strftime('%Y-%m-%d')
        df = pd.read_excel(file,skiprows=[0])
        df['portdate'] = date_
        df = df.rename(columns={'Kode':'portid','Open':'opening','High':'high','Low':'low','Close':'closing','Value':'valuess','Volume':'volume'})
        df = df[['portid','portdate','opening','high','low','closing','valuess','volume']]
        outputxlsx.append(df)
    df_augment = pd.concat(outputxlsx)
    df_augment['portid + portdate'] = df_augment['portid'].astype('str') + ' ' + df_augment['portdate'].astype('str')
    df_augment = df_augment.drop_duplicates(subset='portid + portdate')
    df_augment = df_augment.drop(columns='portid + portdate')
    df_augment = df_augment.sort_values(by='portdate')
    data = data.append(df_augment)
    return data

In [ ]:
# #Augment data dari investdata ke database
# df_all = augment_investdata(data=df_all,file_name=filenames)
# df_all = df_all.reset_index(drop=True)

In [ ]:
# Buat kolom First date sebagai perkiraan tgl IPO
df_all_i_list = []
for i in list(df_all['portid'].unique()):
    df_all_i = df_all[df_all['portid']==i]
    df_all_i = df_all_i.sort_values(by='portdate')
    df_all_i['First Date'] = df_all_i.iloc[[0]]['portdate'].values[0]
    df_all_i['Distance to First Date'] = pd.to_datetime(df_all_i['portdate']) - pd.to_datetime(df_all_i['First Date'])
    df_all_i['Distance to First Date'] = df_all_i['Distance to First Date'].astype('str')
    df_all_i_list.append(df_all_i)
df_all = pd.concat(df_all_i_list)
df_all = df_all.sort_values(by='portdate')
df_all = df_all.reset_index(drop=True)
df_all.head(3)

In [ ]:
print(f"df_all: {np.round(df_all.memory_usage(deep=True).sum()/1e9,2)} Gigabytes")

In [ ]:
df_all = df_all[~df_all['portid'].str.contains(r'-\w+', regex=True)]
print(f"df_all: {np.round(df_all.memory_usage(deep=True).sum()/1e9,2)} Gigabytes")

In [ ]:
#Jika sudah run pembuatan database Laporan Keuangan, mulai dari sini!! Jika belum, buka Scraping_Stockbit.ipynb
income_statement = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/Laporan Keuangan/Income Statement.csv',index_col=0)
income_statement['Kode + Quarter'] = income_statement['Kode'] + ' ' + income_statement['Quarter'].astype(str)

balance_sheet = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/Laporan Keuangan/Balance Sheet.csv',index_col=0)
balance_sheet['Kode + Quarter'] = balance_sheet['Kode'] + ' ' + balance_sheet['Quarter'].astype(str)

cash_flow = pd.read_csv('/Users/arielgraham/Documents/Infovesta Tasks/Laporan Keuangan/Cash Flow.csv',index_col=0)
cash_flow['Kode + Quarter'] = cash_flow['Kode'] + ' ' + cash_flow['Quarter'].astype(str)
cash_flow.head()

In [ ]:
# Check Aggregated values

income_statement_agg_col = list(income_statement.columns[~income_statement.columns.str.contains(r' #', regex=True)])
balance_sheet_agg_col = list(balance_sheet.columns[~balance_sheet.columns.str.contains(r' #', regex=True)])
cash_flow_agg_col = list(cash_flow.columns[~cash_flow.columns.str.contains(r' #', regex=True)])

print(
    f"Income Statements agg: {income_statement_agg_col},\n"
    f"Balance Sheet agg: {balance_sheet_agg_col},\n"
    f"Cash Flow agg: {cash_flow_agg_col}"
)

In [ ]:
# Aggregate here if satisfied

income_statement = income_statement[income_statement_agg_col].dropna(axis=1, how='all')
balance_sheet = balance_sheet[balance_sheet_agg_col].dropna(axis=1, how='all')
cash_flow = cash_flow[cash_flow_agg_col].dropna(axis=1, how='all')

In [ ]:
#Tambah Quarter di lapkeu jika data di TrEquity melebihi tanggal di lapkeu. Tujuannya supaya begitu di-join, data tidak terpotong di quarter terakhir lapkeu.
#Contoh: Quarter terakhir lapkeu adalah Q32024 sedangkan tanggal di TrEquity sampai 10-Okt-2024, berarti tambah Quarter Q4 2024
add_quarter = ['Q1 2025','Q4 2024']     # --> Harus urut jika lebih dari satu!!!! Contoh : ['Q1 2025','Q4 2024']

s_list = []

for s, data_s in income_statement.groupby('Kode', group_keys=False):
    data_q = pd.DataFrame({'Kode': [s] * len(add_quarter), 'Quarter': add_quarter})
    data_q['Kode + Quarter'] = data_q['Kode'] + ' ' + data_q['Quarter']
    data_q = data_q.reindex(columns=income_statement.columns, fill_value=pd.NA).astype(object)  
    data_q = data_q.dropna(axis=1, how='all') # Drop if ALL COL ISNA
    s_list.append(pd.concat([data_q, data_s], ignore_index=True))  
income_statement = pd.concat(s_list, ignore_index=True)

s_list = []
for s, data_s in balance_sheet.groupby('Kode', group_keys=False):
    data_q = pd.DataFrame({'Kode': [s] * len(add_quarter), 'Quarter': add_quarter})
    data_q['Kode + Quarter'] = data_q['Kode'] + ' ' + data_q['Quarter']
    data_q = data_q.reindex(columns=balance_sheet.columns, fill_value=pd.NA).astype(object)  
    data_q = data_q.dropna(axis=1, how='all') # Drop if ALL COL ISNA
    s_list.append(pd.concat([data_q, data_s], ignore_index=True))
balance_sheet = pd.concat(s_list)

s_list = []
for s, data_s in cash_flow.groupby('Kode', group_keys=False):
    data_q = pd.DataFrame({'Kode': [s] * len(add_quarter), 'Quarter': add_quarter})
    data_q['Kode + Quarter'] = data_q['Kode'] + ' ' + data_q['Quarter']
    data_q = data_q.reindex(columns=balance_sheet.columns, fill_value=pd.NA).astype(object)  
    data_q = data_q.dropna(axis=1, how='all') # Drop if ALL COL ISNA
    s_list.append(pd.concat([data_q, data_s], ignore_index=True))
cash_flow = pd.concat(s_list)

lapkeu_sb = income_statement.copy()
lapkeu_sb = lapkeu_sb.merge(balance_sheet.drop(['Kode','Quarter'],axis=1), how='outer', on='Kode + Quarter')
lapkeu_sb = lapkeu_sb.merge(cash_flow.drop(['Kode','Quarter'],axis=1), how='outer', on='Kode + Quarter')
lapkeu_sb['Kode'] = [c[:4] for c in lapkeu_sb['Kode + Quarter']]
lapkeu_sb['Quarter'] = [q[5:7] for q in lapkeu_sb['Kode + Quarter']]
lapkeu_sb['Year'] = [y[8:] for y in lapkeu_sb['Kode + Quarter']]
lapkeu_sb['Quarter Year'] = lapkeu_sb['Quarter'].astype('str') + lapkeu_sb['Year'].astype('str')
lapkeu_sb.insert(2, 'Year', lapkeu_sb.pop('Year'))
lapkeu_sb.insert(3, 'Quarter Year', lapkeu_sb.pop('Quarter Year'))
lapkeu_sb = lapkeu_sb.drop(columns=['Kode + Quarter'])
lapkeu_sb.head(3)

In [ ]:
print(f"lapkeu_sb: {np.round(lapkeu_sb.memory_usage(deep=True).sum()/1e9,2)} Gigabytes")

In [ ]:
#Membuat dictionary berisi Quarter Number (ingat harus urut!! cek dulu di excel)
quarter_num = {}
keys = list(lapkeu_sb['Quarter Year'].unique())
values = range(len(list(lapkeu_sb['Quarter Year'].unique())))
index_ = [i for i in range(len(list(lapkeu_sb['Quarter Year'].unique())))]
for eu,u in enumerate(keys):
    quarter_num[u] = [values[eu]+1]
#pd.DataFrame(quarter_num).T.to_excel('dict.xlsx')

#Buat Quarter Number
lapkeu_sb['Quarter Number'] = [quarter_num[x][0] for x in lapkeu_sb['Quarter Year']]

In [ ]:
#buat function untuk annualizing parameter Income Statement dan Cash Flow
def annualizing(report,annualizing_list):
    for i in range(len(report)):
        if i < (len(report)-4):
            if (report.loc[i,'Kode'] == report.loc[i+3,'Kode']) and ((report.loc[i+1,'Quarter Number']-report.loc[i,'Quarter Number']) == 1) and  ((report.loc[i+2,'Quarter Number']-report.loc[i+1,'Quarter Number']) == 1) and ((report.loc[i+3,'Quarter Number']-report.loc[i+2,'Quarter Number']) == 1):
                if (report.loc[i+1,'Quarter Number'] == ((report.loc[i,'Quarter Number']) + 1)) and (report.loc[i+2,'Quarter Number'] == ((report.loc[i+1,'Quarter Number']) + 1)) and (report.loc[i+3,'Quarter Number'] == ((report.loc[i+2,'Quarter Number']) + 1)):
                    for j in annualizing_list:
                        report.loc[i,f'{j} (annualized)'] = report.loc[i,f'{j}'] + report.loc[i+1,f'{j}'] + report.loc[i+2,f'{j}'] + report.loc[i+3,f'{j}']
                else:
                    for j in annualizing_list:
                        report.loc[i,f'{j} (annualized)'] = ''
            else:
                for j in annualizing_list:
                    report.loc[i,f'{j} (annualized)'] = ''
        else:
            for j in annualizing_list:
                report.loc[i,f'{j} (annualized)'] = ''
    report = report.drop(columns=annualizing_list)
    return report

In [ ]:
# Buat list parameter yg akan di annualizing 
a_list = ['Total Pendapatan','Laba Kotor','Laba Usaha','Laba Sebelum Pajak','Laba Bersih Tahun Berjalan',
          'Arus Kas Dari Aktivitas Operasi','Arus Kas Dari Aktivitas Investasi','Arus Kas Dari Aktivitas Pendanaan']
lapkeu_sb = annualizing(report=lapkeu_sb,annualizing_list=a_list)

In [ ]:
#Buat agar tiap saham punya jumlah Quarter Year yg sama, supaya begitu join dgn price data tidak ada saham yg skip
quarter_year_list = list(lapkeu_sb['Quarter Year'].unique())
lapkeu_sb_i_list = []
for i in lapkeu_sb['Kode'].unique():
    lapkeu_sb_i = lapkeu_sb[lapkeu_sb['Kode']==i]
    quarter_year_list_i = list(lapkeu_sb_i['Quarter Year'])
    quarter_year_list_not_in_i = [x for x in quarter_year_list if x not in quarter_year_list_i]
    df_lapkeu_sb_i = pd.DataFrame(index=list(range(len(quarter_year_list_not_in_i))), columns=lapkeu_sb_i.columns)
    df_lapkeu_sb_i['Quarter Year'] = quarter_year_list_not_in_i
    df_lapkeu_sb_i['Kode'] = lapkeu_sb_i['Kode'].iloc[0]
    df_lapkeu_sb_i['Quarter'] = df_lapkeu_sb_i['Quarter Year'].str[:2]
    df_lapkeu_sb_i['Year'] = df_lapkeu_sb_i['Quarter Year'].str[2:]
    lapkeu_sb_i = pd.concat([lapkeu_sb_i,df_lapkeu_sb_i])
    lapkeu_sb_i = lapkeu_sb_i.sort_values(by=['Year','Quarter'],ascending=False).reset_index(drop=True)
    lapkeu_sb_i_list.append(lapkeu_sb_i)
lapkeu_sb = pd.concat(lapkeu_sb_i_list)
lapkeu_sb = lapkeu_sb.reset_index(drop=True)

#Buat Date From dan Date To
for i in range(len(lapkeu_sb)):
    if lapkeu_sb.loc[i,'Quarter'] == 'Q1':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-01-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-03-31'
    elif lapkeu_sb.loc[i,'Quarter'] == 'Q2':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-04-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-06-30'
    elif lapkeu_sb.loc[i,'Quarter'] == 'Q3':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-07-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-09-30'
    elif lapkeu_sb.loc[i,'Quarter'] == 'Q4':
        quarter_year = lapkeu_sb.loc[i,'Year']
        lapkeu_sb.loc[i,'Date From'] = f'{quarter_year}-10-01'
        lapkeu_sb.loc[i,'Date To'] = f'{quarter_year}-12-31'
    else:
        lapkeu_sb.loc[i,'Date From'] = '' 
        lapkeu_sb.loc[i,'Date To'] = ''

lapkeu_sb_annualized = lapkeu_sb.copy()
lapkeu_sb_annualized.columns = lapkeu_sb_annualized.columns.str.replace(' (annualized)','',regex=False) #semua data selain balance sheet sudah di-anualizedkan


In [ ]:
# Augment Quarter Year yang belum lengkap untuk saham2 yg lapkeu-nya belum dirilis sesuai kuartal terakhir (misal MDKA rilis terakhir Q2 2024, sedangkan saham lain sudah Q3 2024)
# Tujuannya agar begitu di-join, data terakhir lebih lengkap karena data terakhir nantinya akan di lag-2
date_from_list = list(lapkeu_sb_annualized['Date From'].unique())
nearest_date_from = lapkeu_sb_annualized['Date From'].max()
nearest_date_from_ = datetime.strptime(nearest_date_from, '%Y-%m-%d').date()
date_to_list = list(lapkeu_sb_annualized['Date To'].unique())
nearest_date_to = lapkeu_sb_annualized['Date To'].max()
quarter_dict = {'03':'Q1','06':'Q2','09':'Q3','12':'Q4'}
lapkeu_sb_annualized_c_merged_list = []
for c in lapkeu_sb_annualized['Kode'].unique():
    lapkeu_sb_annualized_c = lapkeu_sb_annualized[lapkeu_sb_annualized['Kode']==c]
    last_date_from = lapkeu_sb_annualized_c['Date From'].iloc[0]
    last_date_from_ = datetime.strptime(last_date_from, '%Y-%m-%d').date()
    if last_date_from_ < nearest_date_from_:
        new_date_from_list = list(filter(lambda d: d > last_date_from, date_from_list))
        last_date_to = lapkeu_sb_annualized_c['Date To'].iloc[0]
        new_date_to_list = list(filter(lambda d: d > last_date_to, date_to_list))
        new_lapkeu_df = pd.DataFrame({'Date From':new_date_from_list,'Date To':new_date_to_list})
        new_lapkeu_df['Kode'] = c
        new_lapkeu_df['Year'] = new_lapkeu_df['Date To'].str[:4]
        new_lapkeu_df['Quarter'] = new_lapkeu_df['Date To'].str[5:7]
        new_lapkeu_df['Quarter'] = new_lapkeu_df['Quarter'].map(quarter_dict)
        new_lapkeu_df['Quarter Year'] = new_lapkeu_df['Quarter'].astype('str') + new_lapkeu_df['Year'].astype('str')
        lapkeu_sb_annualized_c_merged = lapkeu_sb_annualized_c.merge(new_lapkeu_df,on=['Kode','Quarter','Year','Quarter Year','Date From','Date To'],how='outer')
        lapkeu_sb_annualized_c_merged = lapkeu_sb_annualized_c_merged.sort_values(by='Date From',ascending=False).reset_index(drop=True)
        lapkeu_sb_annualized_c_merged_list.append(lapkeu_sb_annualized_c_merged)
    else: 
        lapkeu_sb_annualized_c_merged_list.append(lapkeu_sb_annualized_c)
lapkeu_sb_annualized = pd.concat(lapkeu_sb_annualized_c_merged_list)
lapkeu_sb_annualized['Quarter Number'] = [quarter_num[x][0] for x in lapkeu_sb_annualized['Quarter Year']]

In [ ]:
print(f"ends with _x: {list(lapkeu_sb_annualized.columns[lapkeu_sb_annualized.columns.str.endswith('_x')])}")
print(f"ends with _y: {list(lapkeu_sb_annualized.columns[lapkeu_sb_annualized.columns.str.endswith('_y')])}")

In [ ]:
# Pilih LK untuk emiten yang sudah difilter sebelumnya agar ringan dalam menjoin data _x dan _y

lapkeu_list = []
for i in lapkeu_sb_annualized['Kode'].unique():
    lapkeu_i = lapkeu_sb_annualized[lapkeu_sb_annualized['Kode'] == i].copy()
    
    merged_cols = {}  # Dictionary to store merged columns
    
    for col in lapkeu_i.columns:
        if col.endswith('_x'):
            base_col = col[:-2]  # Remove '_x'
            y_col = base_col + '_y'
            if y_col in lapkeu_i.columns:
                merged_cols[base_col] = lapkeu_i[col].fillna(lapkeu_i[y_col])  # Merge _x and _y
            else:
                merged_cols[col] = lapkeu_i[col]  # Keep original column if _y doesn't exist
        elif col.endswith('_y') and col[:-2] not in merged_cols:
            merged_cols[col[:-2]] = lapkeu_i[col]  # Keep _y only if _x didn't exist
        else:
            merged_cols[col] = lapkeu_i[col]  # Keep other columns unchanged

    # Convert dictionary to DataFrame and append
    lapkeu_list.append(pd.DataFrame(merged_cols))

lapkeu_sb_annualized = pd.concat(lapkeu_list)
lapkeu_sb_annualized.head(3)

In [ ]:
print(f"lapkeu_sb: {np.round(lapkeu_sb_annualized.memory_usage(deep=True).sum()/1e9,2)} Gigabytes")

In [ ]:
print(f"ends with _x: {list(lapkeu_sb_annualized.columns[lapkeu_sb_annualized.columns.str.endswith('_x')])}")
print(f"ends with _y: {list(lapkeu_sb_annualized.columns[lapkeu_sb_annualized.columns.str.endswith('_y')])}")

In [ ]:
# Bersihkan double space supaya pas nyari column nanti ga riweuh sorangan

lapkeu_sb_annualized = lapkeu_sb_annualized[[item for item in lapkeu_sb_annualized.columns if not re.search(r'\s{2,}', item)]]

In [ ]:
"""
Parameter yg ada di list_growth_params,
                list_lag_params,
                dan list_selected_params harus sudah annualized !!!
Harus ada di a_list !! (Simplified)
"""

# akun_akun = ['Aset Lancar','Aset Tidak Lancar','Aset','Liabilitas Jangka Pendek','Liabilitas Jangka Panjang','Liabilitas',
#         'Ekuitas','Liabilitas Dan Ekuitas','Kas Dan Setara Kas','Piutang Premi','Piutang Reasuransi','Penempatan Pada Bank Indonesia Dan Bank Lain',
#         'Pinjaman Yang Diberikan','Aset Tetap','Piutang Sewa Pembiayaan','Piutang Pembiayaan Konsumen','Piutang Nasabah','Piutang Lain-Lain',
#         'Portofolio Efek','Aset Tak Berwujud','Investasi Jangka Pendek','Aset Reasuransi','Piutang Usaha','Piutang Premi Dan Reasuransi',
#         'Investasi Jangka Panjang','Goodwill','Piutang Pembiayaan','Obligasi Pemerintah','Investasi Pada Entitas Asosiasi','Deposito Berjangka',
#         'Efek Tersedia Untuk Dijual','Total Pendapatan','Laba Kotor','Laba Usaha','Laba Sebelum Pajak','Laba Bersih Tahun Berjalan',
#         'Arus Kas Dari Aktivitas Operasi','Arus Kas Dari Aktivitas Investasi','Arus Kas Dari Aktivitas Pendanaan']

akun_akun = a_list

list_growth_params = akun_akun 
list_lag_params = akun_akun

list_selected_params = ['Kode','Date','Quarter Year','Saham Beredar'] + [x + ' Growth YoY' for x in akun_akun]

In [ ]:
# Account finder
target = "Total"

matches = [col for col in lapkeu_sb_annualized.columns if target in col]
print(matches)


In [ ]:
# Hitung Growth YoY

list_lapkeu = []
for i in set(list(lapkeu_sb_annualized['Kode'])):
    lapkeu_i = lapkeu_sb_annualized[lapkeu_sb_annualized['Kode']==i]
    lapkeu_i_lag_2 = lapkeu_i[['Kode']]
    for j in list_growth_params:
        lapkeu_i_lag_2.loc[:,f'{j} Lag 2'] = lapkeu_i[f'{j}']
    lapkeu_i_lag_2.loc[:,'Quarter Number'] = lapkeu_i['Quarter Number'] - 2 #Buat lag Quarter Number untuk join
    lapkeu_i_lag_2.drop(columns=['Kode'],inplace=True)
    lapkeu_i = lapkeu_i.merge(lapkeu_i_lag_2, how='outer', on='Quarter Number')
    lapkeu_i_lag_6 = lapkeu_i[['Kode']]
    for j in list_growth_params:
        lapkeu_i_lag_6.loc[:,f'{j} Lag 6'] = lapkeu_i[f'{j}']
    lapkeu_i_lag_6.loc[:,'Quarter Number'] = lapkeu_i.loc[:,'Quarter Number'] - 6 #Buat lag Quarter Number untuk join
    lapkeu_i_lag_6.drop(columns=['Kode'],inplace=True)
    lapkeu_i = lapkeu_i.merge(lapkeu_i_lag_6, how='outer', on='Quarter Number')  
    list_lapkeu.append(lapkeu_i)
lapkeu = pd.concat(list_lapkeu).sort_values(by=['Kode','Quarter Number'])
lapkeu = lapkeu[~lapkeu['Kode'].isna()]
lapkeu = lapkeu.reset_index(drop=True)

# Hitung growth
for j in list_growth_params:
    lapkeu.loc[:,f'{j} Growth YoY'] = (lapkeu[f'{j} Lag 2'].replace('',np.nan).astype('float32')-lapkeu[f'{j} Lag 6'].replace('',np.nan).astype('float32'))/lapkeu[f'{j} Lag 6'].replace('',np.nan).astype('float32')
    lapkeu = lapkeu.drop(columns=[f'{j} Lag 6',f'{j} Lag 2'])

In [ ]:
#Buat Lag 2
list_lapkeu = []
for i in set(list(lapkeu['Kode'])):
    lapkeu_i = lapkeu[lapkeu['Kode']==i]
    #Buat lag 2
    lapkeu_i_lag_2 = lapkeu_i[['Kode']]
    for j in list_lag_params:
        lapkeu_i_lag_2.loc[:,f'{j} Lag 2'] = lapkeu_i[f'{j}'].values
    lapkeu_i_lag_2.loc[:,'Quarter Number'] = lapkeu_i['Quarter Number'] - 2 #Buat lag Quarter Number untuk join
    lapkeu_i_lag_2.drop(columns=['Kode'],inplace=True)
    lapkeu_i = lapkeu_i.merge(lapkeu_i_lag_2, how='outer', on='Quarter Number')
    list_lapkeu.append(lapkeu_i)
lapkeu = pd.concat(list_lapkeu).sort_values(by=['Kode','Quarter Number'])
lapkeu = lapkeu[~lapkeu['Kode'].isna()]
lapkeu = lapkeu.reset_index(drop=True)
for j in list_lag_params:
    lapkeu = lapkeu.drop(columns=f'{j}')
    lapkeu = lapkeu.rename(columns={f'{j} Lag 2':f'{j}'})

In [ ]:
print(f"lapkeu: {np.round(lapkeu.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
      f"lapkeu shape: {lapkeu.shape}")

In [ ]:
year_limit = 2021

df_yl = df_all[pd.to_datetime(df_all['portdate']).apply(lambda x: x.year) >= year_limit]
lapkeu_yl = lapkeu[lapkeu['Year'].astype(int) >= year_limit]

print(f"df_yl: {np.round(df_yl.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
      f"lapkeu shape: {df_yl.shape}")

print(f"lapkeu_yl: {np.round(lapkeu_yl.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
      f"lapkeu shape: {lapkeu_yl.shape}")

In [ ]:
# CRASH MULU DAN ABIS RAM
# Buat kolom date
list_lapkeu = []
for i in set(list(lapkeu_yl['Quarter Year'])):
    lapkeu_i = lapkeu_yl[lapkeu_yl['Quarter Year']==i]
    date_from = lapkeu_i['Date From'].unique()[0]
    date_to = lapkeu_i['Date To'].unique()[0]
    list_lapkeu_k = []
    for k in pd.bdate_range(start=date_from,end=date_to):
        lapkeu_k = lapkeu_i.copy()
        lapkeu_k.loc[:,'Date'] = k
        list_lapkeu_k.append(lapkeu_k)
    lapkeu_k = pd.concat(list_lapkeu_k)
    list_lapkeu.append(lapkeu_k)
lapkeu_all_date = pd.concat(list_lapkeu).reset_index(drop=True)
lapkeu_all_date = lapkeu_all_date.sort_values(by=['Date','Kode']).reset_index(drop=True)

# Merge data dan ubah nama kolom
lapkeu_all_date['Kode + Date'] = lapkeu_all_date['Kode'].astype('str') + ' ' + lapkeu_all_date['Date'].astype('str')
df_yl['Kode + Date'] = df_yl['portid'].astype('str') + ' ' + df_yl['portdate'].astype('str')
database = df_yl.merge(lapkeu_all_date.drop(['Kode','Date'],axis=1), how='left', on='Kode + Date')
database = database.drop('Kode + Date',axis=1)
database = database.rename(columns={'portid':'Kode','portdate':'Date','opening':'Open Price','high':'High Price','low':'Low Price','closing':'Close Price','valuess':'Transaction Value'})

In [ ]:
print(f"database: {np.round(database.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
      f"lapkeu shape: {database.shape}")

In [ ]:
# # TrEquity df_all Q_year
# df_all['Quarter Year'] = pd.to_datetime(df_all['portdate']).dt.to_period('Q').astype(str)
# df_all['Quarter Year'] = df_all['Quarter Year'].str[-1] + df_all['Quarter Year'].str[:4]  # Rearrange format
# df_all['Quarter Year'] = 'Q' + df_all['Quarter Year']  # Add "Q" prefix

# # Match Code
# lapkeu['KodeQuarterYear'] = lapkeu['Kode'] + lapkeu['Quarter Year']
# df_all['KodeQuarterYear'] = df_all['portid'] + df_all['Quarter Year']

In [ ]:
print(f"df_all: {np.round(df_all.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
      f"lapkeu shape: {df_all.shape}")

In [ ]:
lapkeu_all_date = pd.merge(df_yl, lapkeu_yl, on="KodeQuarterYear", how="outer")
lapkeu_all_date = lapkeu_all_date[~lapkeu_all_date['portdate'].isna()]

In [ ]:
lapkeu_all_date[lapkeu_all_date['portid'] == 'ASII'].groupby(['Year', 'Quarter']).last().to_excel("ASII db.xlsx")

In [ ]:


# # CRASH MULU DAN ABIS RAM
# # Buat kolom date
# list_lapkeu = []
# for i in set(list(lapkeu_yl['Quarter Year'])):
#     lapkeu_i = lapkeu_yl[lapkeu_yl['Quarter Year']==i]
#     date_from = lapkeu_i['Date From'].unique()[0]
#     date_to = lapkeu_i['Date To'].unique()[0]
#     list_lapkeu_k = []
#     for k in pd.bdate_range(start=date_from,end=date_to):
#         lapkeu_k = lapkeu_i.copy()
#         lapkeu_k.loc[:,'Date'] = k
#         list_lapkeu_k.append(lapkeu_k)
#     lapkeu_k = pd.concat(list_lapkeu_k)
#     list_lapkeu.append(lapkeu_k)
# lapkeu_all_date = pd.concat(list_lapkeu).reset_index(drop=True)
# lapkeu_all_date = lapkeu_all_date.sort_values(by=['Date','Kode']).reset_index(drop=True)

# # Merge data dan ubah nama kolom
# lapkeu_all_date['Kode + Date'] = lapkeu_all_date['Kode'].astype('str') + ' ' + lapkeu_all_date['Date'].astype('str')
# df_yl['Kode + Date'] = df_yl['portid'].astype('str') + ' ' + df_yl['portdate'].astype('str')
# database = df_yl.merge(lapkeu_all_date.drop(['Kode','Date'],axis=1), how='left', on='Kode + Date')
# database = database.drop('Kode + Date',axis=1)
# database = database.rename(columns={'portid':'Kode','portdate':'Date','opening':'Open Price','high':'High Price','low':'Low Price','closing':'Close Price','valuess':'Transaction Value'})

In [ ]:
# lapkeu_all_date = pd.merge(df_all, lapkeu, on="KodeQuarterYear", how="outer")

In [ ]:
# print(f"lapkeu_all_date: {np.round(lapkeu_all_date.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
#       f"lapkeu shape: {lapkeu_all_date.shape}")

In [ ]:
# YoY_list = list(lapkeu_all_date.columns[lapkeu_all_date.columns.str.contains("YoY")])

# important_columns = ['portid', 'portdate', 'Quarter', 'Year',
#                      'opening', 'high', 'low', 'closing', 'valuess', 'volume',
#                     'Saham Beredar', 'Total Aset', 'Total Liabilitas', 'Total Ekuitas'] + a_list + YoY_list

# database = lapkeu_all_date[important_columns]
# database = database.rename(columns={'portid':'Kode','portdate':'Date','opening':'Open Price','high':'High Price','low':'Low Price','closing':'Close Price','valuess':'Transaction Value'})
# database = database.sort_values(['Kode','Date']).reset_index(drop=True)

In [ ]:
# print(f"database: {np.round(database.memory_usage(deep=True).sum()/1e9,2)} Gigabytes\n"
#       f"lapkeu shape: {database.shape}")

In [ ]:
# # CRASH MULU DAN ABIS RAM
# # Buat kolom date
# list_lapkeu = []
# for i in set(list(lapkeu['Quarter Year'])):
#     lapkeu_i = lapkeu[lapkeu['Quarter Year']==i]
#     date_from = lapkeu_i['Date From'].unique()[0]
#     date_to = lapkeu_i['Date To'].unique()[0]
#     list_lapkeu_k = []
#     for k in pd.bdate_range(start=date_from,end=date_to):
#         lapkeu_k = lapkeu_i.copy()
#         lapkeu_k.loc[:,'Date'] = k
#         list_lapkeu_k.append(lapkeu_k)
#     lapkeu_k = pd.concat(list_lapkeu_k)
#     list_lapkeu.append(lapkeu_k)
# lapkeu_all_date = pd.concat(list_lapkeu).reset_index(drop=True)
# lapkeu_all_date = lapkeu_all_date.sort_values(by=['Date','Kode']).reset_index(drop=True)

# # Merge data dan ubah nama kolom
# lapkeu_all_date['Kode + Date'] = lapkeu_all_date['Kode'].astype('str') + ' ' + lapkeu_all_date['Date'].astype('str')
# df_all['Kode + Date'] = df_all['portid'].astype('str') + ' ' + df_all['portdate'].astype('str')
# database = df_all.merge(lapkeu_all_date.drop(['Kode','Date'],axis=1), how='left', on='Kode + Date')
# database = database.drop('Kode + Date',axis=1)
# database = database.rename(columns={'portid':'Kode','portdate':'Date','opening':'Open Price','high':'High Price','low':'Low Price','closing':'Close Price','valuess':'Transaction Value'})

In [ ]:
# #Only include selected columns
# lapkeu_all_date = lapkeu_all_date.loc[:,list_selected_params] 
# df_all = df_all.loc[:,list_selected_market_params] 

#Merge data dan ubah nama kolom
lapkeu_all_date['Kode + Date'] = lapkeu_all_date['Kode'].astype('str') + ' ' + lapkeu_all_date['Date'].astype('str')
df_all['Kode + Date'] = df_all['portid'].astype('str') + ' ' + df_all['portdate'].astype('str')
database = df_all.merge(lapkeu_all_date.drop(['Kode','Date'],axis=1), how='left', on='Kode + Date')
database = database.drop('Kode + Date',axis=1)
database = database.rename(columns={'portid':'Kode','portdate':'Date','opening':'Open Price','high':'High Price','low':'Low Price','closing':'Close Price','valuess':'Transaction Value'})

In [ ]:
#Ganti saham beredar menjadi saham beredar yang terakhir, untuk perkalian market cap
stocks = pd.DataFrame(database['Kode'].sort_values().unique(),columns=['Kode'])
out_shares_dic = {}
for i in stocks['Kode']:
    d = database[database['Kode']==i]['Saham Beredar'].reset_index(drop=True)
    try:
        out_shares_dic[i] = d.iloc[d.last_valid_index()]
    except TypeError:
        out_shares_dic[i] = np.nan
out_shares = pd.DataFrame(out_shares_dic.items(),columns=['Kode','Last Outstanding Shares'])
database = database.merge(out_shares,how='inner', on='Kode')

In [ ]:
#Buat market cap, log market cap, dan estimated transaction value
database['Market Cap'] = database['Close Price']*database['Last Outstanding Shares']
database['Log Market Cap'] = np.log10(database['Market Cap'])
database['Estimated Transaction Value'] = ((database['High Price']+database['Low Price'])/2)*database['volume']

In [ ]:
database

In [ ]:
#Buat kolom momentum untuk berbagai periode. Asumsi 1 bulan = 20 hari
database_mom = []
for i in set(list(database['Kode'])):
    database_i = database[database['Kode']==i]
    database_i['Momentum 1 Day'] = database_i['Close Price'].pct_change(periods=1)
    database_i['Momentum 3 Day'] = database_i['Close Price'].pct_change(periods=3)
    database_i['Momentum 5 Day'] = database_i['Close Price'].pct_change(periods=5)
    database_i['Momentum 10 Day'] = database_i['Close Price'].pct_change(periods=10)
    database_i['Momentum 1 Month'] = database_i['Close Price'].pct_change(periods=20)
    database_i['Momentum 2 Month'] = database_i['Close Price'].pct_change(periods=40)
    database_i['Momentum 3 Month'] = database_i['Close Price'].pct_change(periods=60)
    database_i['Momentum 6 Month'] = database_i['Close Price'].pct_change(periods=120)
    database_i['Momentum 1 Year'] = database_i['Close Price'].pct_change(periods=240)
    database_i['Momentum 12-2'] = database_i['Close Price'].pct_change(periods=220).shift(20)
    database_i['Total Return 1 Day'] = database_i['Close Price'].pct_change(periods=1).shift(-1)
    database_i['Total Return 3 Day'] = database_i['Close Price'].pct_change(periods=3).shift(-3)
    database_i['Total Return 5 Day'] = database_i['Close Price'].pct_change(periods=5).shift(-5)
    database_i['Total Return 10 Day'] = database_i['Close Price'].pct_change(periods=10).shift(-10)
    database_i['Total Return 1 Month'] = database_i['Close Price'].pct_change(periods=20).shift(-20)
    database_i['Total Return 2 Month'] = database_i['Close Price'].pct_change(periods=40).shift(-40)
    database_i['Total Return 3 Month'] = database_i['Close Price'].pct_change(periods=60).shift(-60)
    database_i['Total Return 6 Month'] = database_i['Close Price'].pct_change(periods=120).shift(-120)
    database_i['Change in Estimated Transaction Value 1 Month'] = database_i['Estimated Transaction Value'].pct_change(periods=20)
    database_mom.append(database_i)
database = pd.concat(database_mom).sort_values(by=['Kode','Date'])

In [ ]:
def rolling_average(df,param,period):
    rolling_dic = {'1 Month':20,'3 Month':60,'6 Month':120,'1 Year':240}
    init_list = ['Kode','Date']
    init_list.append(param)
    df['Kode + Date'] = df['Kode'] + df['Date'].astype('str')
    df_ = df[init_list]
    list_df = []
    for i in set(list(df_['Kode'])):
        df_i = df_[df_['Kode']==i]
        df_i[f'Average {param} {period}'] = df_i[f'{param}'].rolling(rolling_dic[period]).mean()
        list_df.append(df_i)
    df__ = pd.concat(list_df).sort_values(by=['Kode','Date'])
    df__['Kode + Date'] = df__['Kode'].astype('str') + df__['Date'].astype('str')
    df__ = df__.drop_duplicates(subset='Kode + Date')
    df = df.merge(df__[[f'Average {param} {period}','Kode + Date']],on='Kode + Date',how='left')
    return df


In [ ]:
database = rolling_average(df=database,param='Market Cap',period='1 Month')
database = rolling_average(df=database,param='Market Cap',period='6 Month')
database = rolling_average(df=database,param='Market Cap',period='1 Year')
database = rolling_average(df=database,param='Transaction Value',period='1 Month')
database = rolling_average(df=database,param='Transaction Value',period='6 Month')
database = rolling_average(df=database,param='Transaction Value',period='1 Year')

In [ ]:
#Buat PBV
database['PBV'] = database['Market Cap']/database['Total Ekuitas']
database['Average PBV 1 Month'] = database['Average Market Cap 1 Month']/database['Total Ekuitas']
database['Average PBV 6 Month'] = database['Average Market Cap 6 Month']/database['Total Ekuitas']

#Buat Price/Sales
database['Price/Sales'] = database['Market Cap']/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Average Price/Sales 1 Month'] = database['Average Market Cap 1 Month']/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Average Price/Sales 6 Month'] = database['Average Market Cap 6 Month']/database['Total Pendapatan'].replace('',np.nan).astype(float)

#Buat PE Ratio
database['PE Ratio'] = database['Market Cap']/database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)
database['Average PE Ratio 1 Month'] = database['Average Market Cap 1 Month']/database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)

#Buat Price/Operating Cash Flow
database['Price/Operating Cash Flow'] = database['Market Cap']/database['Arus Kas Dari Aktivitas Operasi'].replace('',np.nan).astype(float)
database['Average Price/Operating Cash Flow 1 Month'] = database['Average Market Cap 1 Month']/database['Arus Kas Dari Aktivitas Operasi'].replace('',np.nan).astype(float)

#Buat ROE
database['ROE'] = database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)/database['Total Ekuitas']

#Buat Gross, Operating, dan Net Profit Margin
database['Net Profit Margin'] = database['Laba Bersih Tahun Berjalan'].replace('',np.nan).astype(float)/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Gross Profit Margin'] = database['Laba Kotor'].replace('',np.nan).astype(float)/database['Total Pendapatan'].replace('',np.nan).astype(float)
database['Operating Profit Margin'] = database['Laba Usaha'].replace('',np.nan).astype(float)/database['Total Pendapatan'].replace('',np.nan).astype(float)

#Buat Debt Ratio
database['Debt Ratio'] = database['Total Liabilitas']/database['Total Aset']

# # Tambahkan IHSG
# ihsg = pd.read_excel('IHSG.xls',skiprows=[0])
# ihsg = ihsg[['Tanggal','Nilai']]
# ihsg = ihsg.rename(columns={'Tanggal':'Date','Nilai':'IHSG'})
# ihsg['IHSG 1 Day Return'] = ihsg['IHSG'].pct_change()
# ihsg = ihsg.dropna()
# ihsg['Date'] = ihsg['Date'].astype('str')
# ihsg = ihsg.reset_index(drop=True)
# database = database.merge(ihsg, how='left', on='Date')

# Tambahkan Risk-Free Rate
# rf = pd.read_excel('Suku Bunga Deposito 1 Bulan (LPS).xls',skiprows=[0])
# rf['Rf (annualized)'] = rf['Nilai']*0.01
# rf['Rf'] = ((rf['Nilai']*0.01)+1)**(1/12)-1
# rf = rf.rename(columns={'Tanggal':'Date'})
# rf = rf.drop(columns=['Indeks','Nilai'])
# rf['Month'] = [x.month for x in rf['Date']]
# rf['Year'] = [x.year for x in rf['Date']]
# rf['Month + Year'] = rf['Month'].astype('str') + ' ' + rf['Year'].astype('str')
# rf = rf.drop_duplicates(subset='Month + Year')
# rf = rf.drop(columns='Month + Year')
# rf = rf.reset_index(drop=True)
# for i in range(len(rf)):
#     if rf.loc[i,'Month'] == 1:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-01-31'
#     elif rf.loc[i,'Month'] == 2:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-02-28'
#     elif rf.loc[i,'Month'] == 3:
#         year = lapkeu_sb.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-03-31'
#     elif rf.loc[i,'Month'] == 4:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-04-30'
#     elif rf.loc[i,'Month'] == 5:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-05-31'
#     elif rf.loc[i,'Month'] == 6:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-06-30'
#     elif rf.loc[i,'Month'] == 7:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-07-31'
#     elif rf.loc[i,'Month'] == 8:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-08-31'
#     elif rf.loc[i,'Month'] == 9:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-09-30'
#     elif rf.loc[i,'Month'] == 10:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-10-31'
#     elif rf.loc[i,'Month'] == 11:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-11-30'
#     elif rf.loc[i,'Month'] == 12:
#         year = rf.loc[i,'Year']
#         rf.loc[i,'Date From'] = rf.loc[i,'Date']
#         rf.loc[i,'Date To'] = f'{year}-12-31'
#     else:
#         rf.loc[i,'Date From'] = '' 
#         rf.loc[i,'Date To'] = ''
# list_rf= []
# for i in set(list(rf['Date'])):
#     rf_i = rf[rf['Date']==i]
#     date_from = rf_i['Date From'].unique()[0]
#     date_to = rf_i['Date To'].unique()[0]
#     list_rf_k = []
#     for k in pd.bdate_range(start=date_from,end=date_to):
#         rf_k = rf_i.copy()
#         rf_k.loc[:,'Date'] = k
#         list_rf_k.append(rf_k)
#     try:
#         rf_k = pd.concat(list_rf_k)
#         list_rf.append(rf_k)
#     except ValueError:
#         pass
# rf_all_date = pd.concat(list_rf)
# rf_all_date = rf_all_date.sort_values(by=['Date','Rf'])
# rf_all_date = rf_all_date.drop_duplicates(subset='Date', keep='first').reset_index(drop=True)
# rf_all_date = rf_all_date[['Date','Rf','Rf (annualized)']]
# rf_all_date['Date'] = rf_all_date['Date'].astype('str')
# database = database.merge(rf_all_date, how='left', on='Date')
# database = database.dropna(subset=["Kode"])
# database = database.reset_index(drop=True)


#########

JIKA MAU HITUNG PARAMETER QUALITY IDXVESTA28, BUKA Create_Database_Infovesta28.ipynb !!

###########

In [ ]:
#Tambah Daily Frekuensi (ambil dari Stock Summary BEI, cek 'frekuensi.ipynb')
freq_1 = pd.read_csv('Data Frekuensi 1.csv')
freq_2 = pd.read_csv('Data Frekuensi 2.csv')
freq = pd.concat([freq_1,freq_2])
freq = freq.dropna()
freq['Kode + Date'] = freq['Kode'] + freq['Date'].astype('str')
database['Kode + Date'] = database['Kode'] + database['Date'].astype('str')
database = database.merge(freq[['Frequency','Kode + Date']],on='Kode + Date',how='left')
database = database.drop(columns='Kode + Date')


In [ ]:
database[database['Kode']=='BMRI'].groupby(['Quarter', 'Year']).sum(numeric_only=True)

In [ ]:
size = len(database)
database = database.sort_values(by='Date')
database = database.reset_index(drop=True)
database_1 = database.iloc[:int(size/4)]
database_2 = database.iloc[int(size/4):int(size/2)]
database_3 = database.iloc[int(size/2):int(size*(3/4))]
database_4 = database.iloc[int(size*(3/4)):]


In [ ]:
database_1.to_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 1.csv')
database_2.to_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 2.csv')
database_3.to_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 3.csv')
database_4.to_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 4.csv')


In [ ]:
database_1 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 1.csv')
database_2 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 2.csv')
database_3 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 3.csv')
database_4 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Terbaru/Database Part 4.csv')
database = pd.concat([database_1,database_2,database_3,database_4])

In [ ]:
database[database['Kode']=='BREN']['Average Transaction Value 1 Year']

In [ ]:
database[database['Kode']=='GEMS'].to_excel('GEMS.xlsx')